<a href="https://colab.research.google.com/github/divassya/BigDataAnalysis/blob/main/Spark_transformations_and_actions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Info 
Assignment 1
MET CS777 Big Data Analytics

Faculty - Farshid Alizadeh-Shabdiz, PhD, MBA

Student - Assiya Karatay U95161396 karatay@bu.edu 857-294-7028

In [1]:
!pip install --ignore-installed -q pyspark==3.1.2 

     |████████████████████████████████| 212.4 MB 61 kB/s 
     |████████████████████████████████| 198 kB 49.5 MB/s 


In [2]:
# Set JAVA_HOME and SPARK_HOME
import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
# os.environ["SPARK_HOME"] = "spark-3.0.1-bin-hadoop3.2"

# import findspark
# findspark.init("spark-3.0.1-bin-hadoop3.2")# SPARK_HOME


import sys
import requests
from operator import add
import pandas as pd


from pyspark import SparkConf,SparkContext
from pyspark.streaming import StreamingContext

from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = SparkContext.getOrCreate()

### RDD transformations
#### map(func)
map(func) – returns a new distributed dataset formed by passing each
element of the source through a function “func”
Sample example
Starting with constant array of numbers
Use map and square them
Ø rdd = sc.parallelize([1, 2, 3, 4 ])
Ø rdd.map(lambda x:x*x)
Ø This yields 1, 4, 9, and 16

In [4]:
rdd = sc.parallelize([1, 2, 3, 4 ])
rdd.map(lambda x:x*x).collect()

[1, 4, 9, 16]

#### filter

In [5]:
rdd.filter(lambda x: x%2 == 0).collect()

[2, 4]

#### flatMap(func)

In [6]:
sorted(rdd.flatMap(lambda x:range(1,x)).collect())

[1, 1, 1, 2, 2, 3]

#### union(rdd)

In [7]:
rdd.union(rdd).collect()

[1, 2, 3, 4, 1, 2, 3, 4]

distinct([numPartitions])

In [8]:
sorted(rdd.distinct().collect())

[1, 2, 3, 4]

#### join() # requires expensive shuffle
cogroup()
#### cartesian() can be used for for loop
output of this tranformation will be paralelized and other functions are implemented as in a loop


In [9]:
rdd.cartesian(rdd).collect()

[(1, 1),
 (1, 2),
 (2, 1),
 (2, 2),
 (1, 3),
 (1, 4),
 (2, 3),
 (2, 4),
 (3, 1),
 (3, 2),
 (4, 1),
 (4, 2),
 (3, 3),
 (3, 4),
 (4, 3),
 (4, 4)]

In [ ]:
### 

### RDD Actions
#### reduce(func)
#### collect()
#### count()
#### first()
#### take(num)
#### top(num)



In [10]:
rdd.count()

4

In [11]:
rdd.first()

1

In [12]:
rdd.take(2)

[1, 2]

#### foreach(func) 

In [16]:
def f1(x):
  print(x)
rdd.foreach(f1)

#### countByKey() & countByValue()

In [18]:
rdd2 = sc.parallelize([('a',1),('b', 1),('a',1)])
rdd2.countByKey().items()

dict_items([('a', 2), ('b', 1)])

In [22]:
rdd.countByValue().items()

dict_items([(1, 1), (2, 1), (3, 1), (4, 1)])

#### saveAsTextFile(path)
#### reduceByKey(func)

In [31]:
%timeit rdd2.reduceByKey(lambda a,b: a+b)
# takes top 2 by the counted second value of a pair

11.7 ms ± 236 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [32]:
rdd2.reduceByKey(lambda a,b: a+b).top(2, lambda p: p[1]) 


[('a', 2), ('b', 1)]

#### groupByKey
* data must (key,value) pair
* shuffles everything on same machine and then adds up
* like reduceByKey() but without reducing
* try to avoid because it is expensive
#### sortByKey
do not use it


In [35]:
%timeit rdd2.groupByKey()

11.7 ms ± 202 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### AggregateByKey()
has the 'zero' to init the aggregation
diff agg functions can be used